In [44]:
# 모델 구현 시간 체크
import time
# 시작시간 체크
start = time.time()

# 패키지 로딩
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글폰트 적용
plt.rcParams['font.family'] = 'Malgun Gothic'

# 수입 데이터 로딩 (가상데이터)
df_org = pd.read_csv('train.csv')
#쓸모없는 열 삭제
del df_org['Unnamed: 0']

### Nan 값, 데이터셋  처리 

In [45]:
df = df_org.fillna('없음')
df = df.drop(['핵심적발','신고번호','신고일자','검사결과코드'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76837 entries, 0 to 76836
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   통관지세관부호   76837 non-null  int64  
 1   신고인부호     76837 non-null  object 
 2   수입자부호     76837 non-null  object 
 3   해외거래처부호   76837 non-null  object 
 4   특송업체부호    76837 non-null  object 
 5   수입통관계획코드  76837 non-null  object 
 6   수입신고구분코드  76837 non-null  object 
 7   수입거래구분코드  76837 non-null  int64  
 8   수입종류코드    76837 non-null  int64  
 9   징수형태코드    76837 non-null  int64  
 10  신고중량(KG)  76837 non-null  float64
 11  과세가격원화금액  76837 non-null  float64
 12  운송수단유형코드  76837 non-null  int64  
 13  반입보세구역부호  76837 non-null  int64  
 14  HS10단위부호  76837 non-null  int64  
 15  적출국가코드    76837 non-null  object 
 16  원산지국가코드   76837 non-null  object 
 17  관세율구분코드   76837 non-null  object 
 18  관세율       76837 non-null  float64
 19  우범여부      76837 non-null  int64  
dtypes: float64(3), int64(8), obj

In [46]:
discrete_columns = ['통관지세관부호',
       '신고인부호', '수입자부호', '해외거래처부호', '특송업체부호', '수입통관계획코드', 
       '수입신고구분코드', '수입거래구분코드','수입종류코드', '징수형태코드', 
       '운송수단유형코드', '반입보세구역부호', 'HS10단위부호', '적출국가코드', 
       '원산지국가코드', '관세율구분코드']

for var in discrete_columns:
    df[var] = df[var].astype(str)

In [47]:
numeric_columns = ['신고중량(KG)', '과세가격원화금액', '관세율', '우범여부']

for var in ['신고중량(KG)', '과세가격원화금액']:
    df[var] = df[var].apply(lambda x: np.log1p(x))

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76837 entries, 0 to 76836
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   통관지세관부호   76837 non-null  object 
 1   신고인부호     76837 non-null  object 
 2   수입자부호     76837 non-null  object 
 3   해외거래처부호   76837 non-null  object 
 4   특송업체부호    76837 non-null  object 
 5   수입통관계획코드  76837 non-null  object 
 6   수입신고구분코드  76837 non-null  object 
 7   수입거래구분코드  76837 non-null  object 
 8   수입종류코드    76837 non-null  object 
 9   징수형태코드    76837 non-null  object 
 10  신고중량(KG)  76837 non-null  float64
 11  과세가격원화금액  76837 non-null  float64
 12  운송수단유형코드  76837 non-null  object 
 13  반입보세구역부호  76837 non-null  object 
 14  HS10단위부호  76837 non-null  object 
 15  적출국가코드    76837 non-null  object 
 16  원산지국가코드   76837 non-null  object 
 17  관세율구분코드   76837 non-null  object 
 18  관세율       76837 non-null  float64
 19  우범여부      76837 non-null  int64  
dtypes: float64(3), int64(1), obj

In [49]:
df.shape

(76837, 20)

In [50]:

df['특송업체발송비율'] = df['관세율']
df['특송업체발송비율'] = df['특송업체발송비율'].apply(lambda x: np.log1p(x))
df_z = df['특송업체부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df.loc[df['특송업체부호'] == du[i] , '특송업체발송비율'] = dc[i]

In [51]:
i = '통관지세관비율'
j = '통관지세관부호'
df[i] = df['관세율']
df[i] = df[i].apply(lambda x: np.log1p(x))

df_z = df[j].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for c in range(df_z.shape[0]):
    df.loc[df[j] == du[c] , i] = dc[c]

In [55]:
i = '해외거래처비율'
j = '해외거래처부호'
df[i] = df['관세율']
df[i] = df[i].apply(lambda x: np.log1p(x))

df_z = df[j].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for c in range(df_z.shape[0]):
    df.loc[df[j] == du[c] , i] = dc[c]
df = df.drop([j], axis=1)

In [57]:
i = '운송수단비율'
j = '운송수단유형코드'
df[i] = df['관세율']
df[i] = df[i].apply(lambda x: np.log1p(x))

df_z = df[j].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for c in range(df_z.shape[0]):
    df.loc[df[j] == du[c] , i] = dc[c]
df = df.drop([j], axis=1)

In [63]:
df.head(10)

,통관지세관부호,신고인부호,수입자부호,특송업체부호,수입통관계획코드,수입신고구분코드,수입거래구분코드,수입종류코드,징수형태코드,신고중량(KG),과세가격원화금액,반입보세구역부호,HS10단위부호,적출국가코드,원산지국가코드,관세율구분코드,관세율,우범여부,특송업체발송비율,통관지세관비율,해외거래처비율,운송수단비율
0,121,2O5A2,82ZHWNL,TQ18AK,D,B,15,23,43,4.844187,8.593963,2106003,8481900000,US,US,A,8.0,1,0.014719,0.001783,0.342192,0.467158
1,30,305K5,5IS70LE,없음,C,B,11,21,11,10.303820,13.258812,4077010,2106909099,US,US,A,8.0,0,0.673256,0.117782,0.342192,0.482658
2,20,CGMT6,GJ5KBL3,없음,D,B,11,21,18,10.067242,10.862759,4077007,6307909000,US,US,A,10.0,0,0.673256,0.160782,0.000104,0.482658
3,40,QWUTG,PBYW02T,없음,C,B,94,21,43,9.429564,14.388527,4077106,6505009090,CN,CN,A,8.0,0,0.673256,0.507659,0.342192,0.482658
4,30,0X1CO,MCX0GJB,없음,C,B,11,21,43,9.661015,15.987683,2006075,6204320000,CN,CN,FCN1,5.2,1,0.673256,0.117782,0.000065,0.482658
5,40,QV6WA,W1TC7ZF,없음,D,B,11,21,18,8.966854,11.516296,3077102,9506910000,US,US,A,8.0,0,0.673256,0.507659,0.001171,0.467158
6,40,BTMNQ,I0J35SK,PAVJZL,D,B,11,21,11,9.692655,0.000000,2002079,3915909000,JP,JP,C,6.5,0,0.036415,0.507659,0.001705,0.482658
7,40,9TJBD,MK4UK01,없음,D,B,87,21,11,10.320581,15.432589,4077101,5515119000,CN,CN,FCN1,6.0,0,0.673256,0.507659,0.001796,0.467158
8,20,ZAV8X,J5VI59Y,없음,C,B,91,21,11,9.164652,14.491094,4077008,8504403010,CN,CN,CIT,0.0,1,0.673256,0.160782,0.002980,0.030532
9,30,RDDA3,IF8PWQX,없음,B,B,11,21,11,9.966852,18.843545,3077102,2941909099,IN,IN,FIN1,0.0,0,0.673256,0.117782,0.002225,0.467158


In [59]:
df.columns

Index(['통관지세관부호', '신고인부호', '수입자부호', '특송업체부호', '수입통관계획코드', '수입신고구분코드',
       '수입거래구분코드', '수입종류코드', '징수형태코드', '신고중량(KG)', '과세가격원화금액', '반입보세구역부호',
       'HS10단위부호', '적출국가코드', '원산지국가코드', '관세율구분코드', '관세율', '우범여부', '특송업체발송비율',
       '통관지세관비율', '해외거래처비율', '운송수단비율'],
      dtype='object')

In [10]:
dc[0]

0.673256